<a href="https://colab.research.google.com/github/HarishSingh1981/LearningNeuralNetwork/blob/main/S5/Assign_5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [4]:
#attempt 2
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.convblk1 = nn.Sequential(nn.Conv2d(1, 10, 3),
                                      nn.BatchNorm2d(10),
                                      nn.ReLU())                              
        self.convblk2 = nn.Sequential(nn.Conv2d(10, 10, 3),
                                      nn.BatchNorm2d(10),
                                      nn.ReLU())
        self.transitionBlock1 = nn.MaxPool2d(2,2)
        self.convblk3 = nn.Sequential(nn.Conv2d(10,32,3),
                                      nn.BatchNorm2d(32),
                                      nn.ReLU())
        
        self.convblk4 = nn.Sequential(nn.Conv2d(32,10,3,padding=1),
                                      nn.BatchNorm2d(10),
                                      nn.ReLU())
        self.convblk5 = nn.Sequential(nn.Conv2d(10,10,3),
                                      nn.BatchNorm2d(10),
                                      nn.ReLU())

        self.convblk6 = nn.Sequential(nn.Conv2d(10,20,3),
                                      nn.BatchNorm2d(20),
                                      nn.ReLU())
        self.convblk7 = nn.Sequential(nn.Conv2d(20,10,1),
                                      nn.BatchNorm2d(10),
                                      nn.ReLU())
        self.avgPoolblk = nn.AvgPool2d(7,7)
    def forward(self, x):
      #maxpool must be used at least after 2 convolution and sud be as far as possible from last layer
        x = self.convblk1(x) #input 1x28x28 -? OUtput 10x26x26? RF 3
        x = self.convblk2(x) #input 10x26x26 -? OUtput 10x24x24? RF 5
        x = self.convblk3(x) #input 10x24x24 -? OUtput 32x22x22? RF 7
        x = self.transitionBlock1(x) #input 32x22x22 -? OUtput 32x11x11? RF 8
        x = self.convblk4(x) #input 32x11x11 -? OUtput 10x11x11? RF 12
#        x = self.transitionBlock2(x)
        x = self.convblk5(x) #input 10x11x11 -? OUtput 10x9x9? RF 16
        x = self.convblk6(x) #input 10x9x9 -? OUtput 20x7x7? RF 20
        x = self.convblk7(x) #input 20x7x7 -? OUtput 10x7x7? RF 20
        x = self.avgPoolblk(x) #input 10x7x7 -? OUtput 10x1x1? RF 32
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [5]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
       BatchNorm2d-2           [-1, 10, 26, 26]              20
              ReLU-3           [-1, 10, 26, 26]               0
            Conv2d-4           [-1, 10, 24, 24]             910
       BatchNorm2d-5           [-1, 10, 24, 24]              20
              ReLU-6           [-1, 10, 24, 24]               0
            Conv2d-7           [-1, 32, 22, 22]           2,912
       BatchNorm2d-8           [-1, 32, 22, 22]              64
              ReLU-9           [-1, 32, 22, 22]               0
        MaxPool2d-10           [-1, 32, 11, 11]               0
           Conv2d-11           [-1, 10, 11, 11]           2,890
      BatchNorm2d-12           [-1, 10, 11, 11]              20
    

<ipython-input-4-d4d71fc22d15>:44: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [6]:
import torchvision


torch.manual_seed(1)
batch_size = 128

#kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}
kwargs = dict(num_workers= 4, pin_memory= True if use_cuda else False,shuffle=True,batch_size=batch_size)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,)),
                        transforms.RandomRotation(degrees=7)
                    ])),**kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                        #transforms.RandomRotation(degrees=7)
                    ])),**kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    train_loss = 0
    correct = 0
    processed=0
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        train_loss +=loss
        pred = output.argmax(dim=1,keepdim=True)
        #need to understand what does it do
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accurary={100*correct/processed:0.2f}')

    #train_loss /= len(train_loader.dataset)
   # print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
     #   train_loss, correct, len(test_loader.dataset),
      #  100. * correct / len(train_loader.dataset)))
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
from torch.optim.lr_scheduler import StepLR
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
for epoch in range(15):
    print(f'Epoch--> {epoch+1}')
    train(model, device, train_loader, optimizer, epoch)
    print("Current learning rate:", optimizer.param_groups[0]['lr'])
    test(model, device, test_loader)

Epoch--> 1


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-4-d4d71fc22d15>:44: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.1136191114783287 batch_id=468 Accurary=93.41: 100%|██████████| 469/469 [00:30<00:00, 15.17it/s]

Current learning rate: 0.1



Test set: Average loss: 0.0663, Accuracy: 9819/10000 (98.19%)

Epoch--> 2


loss=0.034675370901823044 batch_id=468 Accurary=97.91: 100%|██████████| 469/469 [00:32<00:00, 14.64it/s]

...

loss=0.011762890964746475 batch_id=468 Accurary=99.23: 100%|██████████| 469/469 [00:33<00:00, 13.85it/s]


Current learning rate: 0.1

Test set: Average loss: 0.0230, Accuracy: 9927/10000 (99.27%)

Epoch--> 15


loss=0.09446754306554794 batch_id=468 Accurary=99.21: 100%|██████████| 469/469 [00:32<00:00, 14.31it/s]

Current learning rate: 0.1



Test set: Average loss: 0.0248, Accuracy: 9913/10000 (99.13%)

